# Provision an Azure SQL Database to multiple local copies [for Dev]

  
Demo PowerShell Script for PASS Summit Demo

Author: Chris Unwin - Redgate Software - chris.unwin@red-gate.com

Last Modified Date: 29/10/2019 

Pre-requisites: Modules: Az (https://www.powershellgallery.com/packages/Az/), dbatools (https://dbatools.io/)

Start export of Azure SQL DB

In [6]:
Connect-AzAccount
Set-AzContext -SubscriptionId "a36be632-e20c-48fd-8af0-ba5b2c623951"
$AzureUser = "chris.unwin"
$AzureUserPassword  = ConvertTo-SecureString "Manchestercity1" -asplaintext -force
$ServerName = "dmproduction"
$StorageURI = "https://dmstoragechris.blob.core.windows.net/dmblobstore/DMDatabase_Production"+ [datetime]::Today.ToString('yyyy-MM-dd') +".bacpac"
$ExportRequest = New-AzSqlDatabaseExport -DatabaseName "DMDatabase_Production" `
-ServerName $ServerName `
-StorageKeyType "storageaccesskey" `
-StorageKey "LdkmJkNntY+c4Mu2++He2D//QavZFL7b0I50tkJYsESujoGpXhNpF2eRoHNc3ZoW9iRS5QIYlfa2oBrKQiFBgQ==" `
-StorageUri $StorageURI `
-ResourceGroupName "DMDb" `
-AdministratorLogin $AzureUser `
-AdministratorLoginPassword $AzureUserPassword
$ExportStatus = Get-AzSqlDatabaseImportExportStatus -OperationStatusLink $ExportRequest.OperationStatusLink
[Console]::Write("Exporting")
while ($ExportStatus.Status -eq "InProgress")
{
    $ExportStatus = Get-AzSqlDatabaseImportExportStatus -OperationStatusLink $ExportRequest.OperationStatusLink
    [Console]::Write(".")
    Start-Sleep -s 30
}
[Console]::WriteLine("")
$ExportStatus